In [1]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 107MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 183MB/s]


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler

In [3]:
train_df = pd.read_csv("train_data.csv")
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].to_numpy()

In [4]:
test_df = pd.read_csv("test_data.csv")
X_test = test_df.drop(['target'], axis=1)
y_test = test_df['target'].to_numpy()

In [5]:
X_train.shape

(2842, 110)

# TF-IDF

In [6]:
X_train_tfidf= X_train.drop(['clean_text'], axis=1).to_numpy()
X_test_tfidf = X_test.drop(['clean_text'], axis=1).to_numpy()

In [7]:
tfidf = TfidfVectorizer()

In [8]:
tfidf.fit(pd.concat([X_train['clean_text'], X_test['clean_text']],ignore_index=True))

TfidfVectorizer()

In [9]:
train_tfidf = tfidf.transform(X_train['clean_text']).toarray()

In [10]:
X_train_tfidf = np.concatenate((X_train_tfidf, train_tfidf), axis=1)

In [11]:
test_tfidf = tfidf.transform(X_test['clean_text']).toarray()

In [12]:
X_test_tfidf = np.concatenate((X_test_tfidf, test_tfidf), axis=1)

In [13]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(2842, 10341)
(711, 10341)


# Training-Testing Phase

In [14]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [21]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [ ]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(alpha=0.4), scaler.fit_transform(X_train_tfidf), scaler.fit_transform(X_test_tfidf), y_train, y_test)

{'Accuracy': 0.7285513361462729,
 'Precision': 0.6818181818181818,
 'Recall': 0.8260869565217391,
 'F1-Score': 0.747051114023591}

In [ ]:
training_and_prediction(GaussianNB(), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.6019690576652602,
 'Precision': 0.5771144278606966,
 'Recall': 0.672463768115942,
 'F1-Score': 0.6211512717536813}

In [ ]:
training_and_prediction(BernoulliNB(alpha=0.3), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.7454289732770746,
 'Precision': 0.7204301075268817,
 'Recall': 0.7768115942028986,
 'F1-Score': 0.7475592747559275}

## SVM

In [ ]:
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'gamma': [0.001, 0.01, 0.1, 1],
#     'kernel': ['sigmoid', 'rbf', 'poly']
# }

In [ ]:
# grid_search = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
# grid_search.fit(X_train_tfidf, y_train)


In [ ]:
# grid_search.best_estimator_

In [16]:
training_and_prediction(SVC(kernel='poly'), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.7609001406469761,
 'Precision': 0.7272727272727273,
 'Recall': 0.8115942028985508,
 'F1-Score': 0.767123287671233}

## Random Forest

In [ ]:
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [None, 10, 20],
#     'min_samples_split': [2, 5, 10],
# }

# grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
# grid_search.fit(X_train_tfidf, y_train)
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)

In [18]:
training_and_prediction(RandomForestClassifier(), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.7482419127988749,
 'Precision': 0.7292817679558011,
 'Recall': 0.7652173913043478,
 'F1-Score': 0.746817538896747}

## Logistic Regression

In [ ]:
# # Define the parameter grid
# param_grid = {
#     'penalty': ['l1', 'l2'],
#     'C': [0.001, 0.01, 0.1, 1, 10, 100],
#     'solver': ['liblinear', 'saga']
# }

# # Create a Logistic Regression classifier
# logistic_reg = LogisticRegression()

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=logistic_reg, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# # Perform grid search
# grid_search.fit(X_train_tfidf, y_train)

# # Print the best parameters and best score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)


In [19]:
training_and_prediction(LogisticRegression(max_iter=5000, solver='saga'), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.7665260196905767,
 'Precision': 0.7507002801120448,
 'Recall': 0.7768115942028986,
 'F1-Score': 0.7635327635327636}

## Gredient Boosting

In [ ]:
# # Define the parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.1, 0.2],
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 4, 5],
#     'subsample': [0.5, 0.7, 1.0],
#     'colsample_bytree': [0.5, 0.7, 1.0]
# }

# # Create an XGBoost classifier
# xgb_classifier = XGBClassifier()

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# # Perform grid search
# grid_search.fit(X_train_tfidf, y_train)

# # Print the best parameters and best score
# print("Best Parameters:", grid_search.best_params_)
# print("Best Score:", grid_search.best_score_)

In [ ]:
training_and_prediction(XGBClassifier(), X_train_tfidf, X_test_tfidf, y_train, y_test)

{'Accuracy': 0.7623066104078763,
 'Precision': 0.7573099415204678,
 'Recall': 0.7507246376811594,
 'F1-Score': 0.7540029112081513}

## Deep Learning Techniques with Word2vec

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

### ANN

In [23]:
def ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(Dense(num_neurons_layer1, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
    model_dense.add(Dropout(dropout_rate))
    model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_tfidf, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_tfidf, y_test))
    y_pred_dense = model_dense.predict(X_test_tfidf)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]

In [24]:
for num_neurons_layer1 in [256, 128, 64]:
  for num_neurons_layer2 in [128, 64, 32]:
    for dropout_rate in [0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

23/23 [==============================] - 0s 9ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.7580872011251758, 'Precision': 0.7270341207349081, 'Recall': 0.8028985507246377, 'F1-Score': 0.7630853994490359}]
23/23 [==============================] - 0s 7ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7735583684950773, 'Precision': 0.7323232323232324, 'Recall': 0.8405797101449275, 'F1-Score': 0.7827260458839406}]
23/23 [==============================] - 0s 7ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.759493670886076, 'Precision': 0.7485714285714286, 'Recall': 0.7594202898550725, 'F1-Score': 0.753956834532374}]
23/23 [==============================] - 0s 7ms/step
[{'num_neurons_layer1': 256, 'num_neurons_layer2': 128, 'dropout_rate': 0.4}, {'Accuracy': 0.7637130801687764, 'Precision': 0.7196029776674938, 'Recall': 0.8405797101449